稳定版本，仅能一次分析一个文件夹

In [24]:
import os
import json

# 定义文件路径
folder = 5
json_folder = f'F:/workspace/data_labeling/data/{folder}'
output_file = f'F:/workspace/data_labeling/output/{folder}.txt'

# 定义图片命名规则和顺序
image_files = [f"zn{i}.jpg" for i in range(1, 51)] + [f"zp{i}.jpg" for i in range(51)]
total_images = len(image_files)

# 初始化时间步长和z轴范围
time_step = 0.10101010101010101
z_values = [i * 0.02 for i in range(50)] + [0] + [-i * 0.02 for i in range(1, 51)]

# 函数：归一化坐标
def normalize_coordinate(point, image_size):
    return point / image_size

# 函数：解析JSON文件并提取坐标
def parse_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    points = data['shapes'][0]['points']
    x_coords = [point[0] for point in points]
    y_coords = [point[1] for point in points]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    x_norm = normalize_coordinate(x_center, data['imageWidth'])
    y_norm = normalize_coordinate(y_center, data['imageHeight'])
    return x_norm, y_norm

# 读取所有标注并生成输出
with open(output_file, 'w') as f_out:
    for idx, image_file in enumerate(image_files):
        time = (idx + 1) * time_step
        z_coord = z_values[idx]
        
        json_file = os.path.join(json_folder, image_file.replace('.jpg', '.json'))
        if os.path.exists(json_file):
            x_norm, y_norm = parse_json(json_file)
        else:
            if image_file.startswith('zn'):
                ref_json_file = os.path.join(json_folder, 'zn25.json')
            else:
                ref_json_file = os.path.join(json_folder, 'zp25.json')
            x_norm, y_norm = parse_json(ref_json_file)
            
        f_out.write(f"{format(time, '.16f')},{format(x_norm, '.16f')},{format(y_norm, '.16f')},{format(z_coord, '.16f')}\n")



以下是有bug版本

In [14]:
import os
import json

# 定义顶级文件夹路径
root_folder  = 'F:/workspace/srp_imagelabeling/data/'

# 定义图片命名规则和顺序
image_files = [f"zn{i}.jpg" for i in range(1, 51)] + [f"zp{i}.jpg" for i in range(51)]
total_images = len(image_files)

# 初始化时间步长和z轴范围
time_step = 0.10101010101010101
z_values = [i * 0.02 for i in range(50)] + [0] + [-i * 0.02 for i in range(1, 51)]

# 函数：归一化坐标
def normalize_coordinate(point, image_size):
    return point / image_size

# 函数：解析JSON文件并提取坐标
def parse_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    points = data['shapes'][0]['points']
    x_coords = [point[0] for point in points]
    y_coords = [point[1] for point in points]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2
    x_norm = normalize_coordinate(x_center, data['imageWidth'])
    y_norm = normalize_coordinate(y_center, data['imageHeight'])
    return x_norm, y_norm

# 初始化计数器
successful_folders = 0
failed_folders = 0

# 遍历所有文件夹
for folder_name in sorted(os.listdir(root_folder)):
    json_folder = os.path.join(root_folder, folder_name)
    if not os.path.isdir(json_folder):
        continue
    
    output_file = os.path.join(root_folder, f"{folder_name}.txt")
    folder_parsed = False
    
    with open(output_file, 'w') as f_out:
        for idx, image_file in enumerate(image_files):
            time = (idx + 1) * time_step
            z_coord = z_values[idx]
            
            json_file = os.path.join(json_folder, image_file.replace('.jpg', '.json'))
            if os.path.exists(json_file):
                x_norm, y_norm = parse_json(json_file)
                folder_parsed = True
            else:
                if image_file.startswith('zn'):
                    ref_json_file = os.path.join(json_folder, 'zn25.json')
                else:
                    ref_json_file = os.path.join(json_folder, 'zp25.json')
                
                if os.path.exists(ref_json_file):
                    x_norm, y_norm = parse_json(ref_json_file)
                    folder_parsed = True
            
            f_out.write(f"{format(time, '.16f')},{format(x_norm, '.16f')},{format(y_norm, '.16f')},{format(z_coord, '.16f')}\n")
    
    if folder_parsed:
        successful_folders += 1
    else:
        failed_folders += 1

# 输出总结信息
print(f"成功解析的文件夹数量: {successful_folders}")
print(f"未解析的文件夹数量: {failed_folders}")


成功解析的文件夹数量: 1
未解析的文件夹数量: 98
